# 装饰器

In [5]:
# 无法装饰类函数  
class tracer:  
    def __init__(self, func):  
        self.calls = 0  
        self.func = func  
  
    def __call__(self, *args):  
        self.calls += 1  
        print(f'call {self.calls} to {self.func}')  
        self.func(*args)

@tracer  
def spam(a, b, c):  
    print(a + b + c)  
# 等价于
# spam = tracer(spam)

print(spam(1, 2, 3))  
print(spam('a', 'b', 'c'))

call 1 to <function spam at 0x111bc89a0>
6
None
call 2 to <function spam at 0x111bc89a0>
abc
None


按照这种⽅式编程的时候，被装饰的⽅法重绑定到装饰器类的⼀个实例，⽽不是一个简单函数。这⼀点带来的问题是，当装饰器的_ca11_⽅法随后运⾏的时候，其中的se1f接收decorator 类实例，并且类C的实例不会包含到⼀个 *args 中。这使得把调⽤分派给最初的⽅法变得不可能，即保持了最初的⽅法函数的装饰器对象，但是没有实例传递给它。

In [31]:
class decorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self.func(*args)
    
class C:
    @decorator
    def method(self, x, y):
        print(x + y)

init <function C.method at 0x112a13240>


In [30]:
X = C()
X.method(2,5)

args: (2, 5), kwargs: {}
call 1 to <function C.method at 0x112a104a0>


TypeError: C.method() missing 1 required positional argument: 'y'

In [28]:
X.method(1,2,5)

args: (1, 2, 5), kwargs: {}
call 2 to <function C.method at 0x111d03880>
1 2 5


In [32]:
Z = C()
Z.method(1,2,5)

args: (1, 2, 5), kwargs: {}
call 1 to <function C.method at 0x112a13240>
1 2 5


为了⽀持函数和⽅法，嵌套函数这⼀替代⽅案将有更好的效果

In [34]:
def decorator(func):
    def wrapper(*args):
        return func(*args)
    return wrapper
    
class C:
    @decorator
    def method(self, x, y):
        print(x + y)
        
@decorator
def method(x, y):
    print(x + y)

In [35]:
X = C()
X.method(2,5)

7


In [36]:
method(2,5)

7


## 类装饰器

在这个例子中，装饰器把类的名称重新绑定到另一个类，这个类在外层作用域中保持了最初的类，并且当调用的时候，这个类创建并嵌入了最初类的一个实例。当之后从该实例获取一个属性的时候，包装器的 _getattr_拦截了它，并且将其委托给最初的类的嵌入实例。
此外，每个被装饰的类都创建一个新的作用域，它记住了最初的类。在本章后面，我们将用一些更有用的代码来充实这个例子。

In [48]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args, **kwargs):
            self.wrapped = cls(*args, **kwargs)
        def __getattr__(self, attrname):
            return getattr(self.wrapped, attrname)
    return Wrapper

@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'nice'

In [49]:
x = C(6,7)

In [50]:
print(x.attr, x.wrapped.attr)

nice nice


In [51]:
print(x.x, x.y)

AttributeError: 'C' object has no attribute 'x'

In [52]:
[x for x in dir(x) if not x.startswith('_')]

['wrapped']

In [53]:
[x for x in dir(x.wrapped) if not x.startswith('_')]

['attr']

支持多个实例

没有能够处理给定类的多个实例,每次实例创建调⽤都覆盖了上⼀次保存的实例

In [57]:
class Decorator:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        self.wrapped = self.func(*args, **kwargs)
    def __getattr__(self, item):
        return getattr(self.wrapped, item)

@Decorator
class C: ...

In [58]:
x= C()
print(id(x))
y = C()
print(id(x), id(y))

4340489968
4340489968 4340489968


In [59]:
x is y

True

装饰器嵌套

In [60]:
def d1(func): return lambda: 'd1'+ func()
def d2(func): return lambda: 'd2'+ func()
def d3(func): return lambda: 'd3'+ func()

@d1
@d2
@d3
def f():
    return 'f'

print(f())

d1d2d3f


类实例属性

In [64]:
# 无法装饰类函数  
class tracer:  
    def __init__(self, func):  
        self.calls = 0  
        self.func = func  
  
    def __call__(self, *args, **kwargs):  
        self.calls += 1  
        print(f'call {self.calls} to {self.func.__name__}')  
        self.func(*args, **kwargs)

@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    
# 等价于
# spam = tracer(spam)

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
6
call 2 to spam
abc
call 1 to eggs
eggs
call 2 to eggs
eggs


外部作用域和非局部变量

In [65]:
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print(f'call {calls} to {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
6
call 2 to spam
abc
call 1 to eggs
eggs
call 2 to eggs
eggs


函数属性

In [66]:
def tracer(func):
    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        print(f'call {wrapper.calls} to {func.__name__}')
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
6
call 2 to spam
abc
call 1 to eggs
eggs
call 2 to eggs
eggs


In [91]:
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls+=1
        print(f'call {calls} to {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

class C:
    @tracer
    def method1(self, x, y):
        print(x + y)
    @tracer
    def method2(self, x, y):
        print(x * y)

In [92]:
x = C()
x.method1(1,2)
x.method2(10,25)
x.method2(10,2)
y = C()
y.method1(1,5)
y.method2(10,2)

call 1 to method1
3
call 1 to method2
250
call 2 to method2
20
call 2 to method1
6
call 3 to method2
20


In [90]:
@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
6
call 2 to spam
abc
call 1 to eggs
eggs
call 2 to eggs
eggs


## 使⽤描述符装饰⽅法

In [106]:
class tracer:
    """一个追踪装饰器类，用于追踪函数调用的次数。"""
    def __init__(self, func):
        self.calls = 0
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print()
        self.calls += 1
        print(f'call {self.calls} to {self.func.__name__}')
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner):
        print(f'getting {self.func.__name__}')
        return wrapper(self, instance)
        
        
class wrapper:
    """wrapper类用于包装tracer类实例，以支持方法绑定。"""
    def __init__(self, desc, subj):
        """
            desc (tracer): 被包装的tracer实例。
            subj (object): 类的实例。
        """
        self.desc = desc
        self.subj = subj
        
    def __call__(self, *args, **kwargs):
        """使wrapper实例可调用，通过tracer实例调用被装饰函数。"""
        return self.desc(self.subj, *args, **kwargs)


In [107]:
class C:
    @tracer
    def method1(self, x, y):
        print(x + y)

    @tracer
    def method2(self, x, y):
        print(x * y)


In [108]:

x = C()
x.method1(1, 2)
x.method2(10, 25)
x.method2(10, 2)
y = C()
y.method1(1, 5)
y.method2(10, 2)

getting method1

call 1 to method1
3
getting method2

call 1 to method2
250
getting method2

call 2 to method2
20
getting method1

call 2 to method1
6
getting method2

call 3 to method2
20


In [100]:
@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
6
call 2 to spam
abc
call 1 to eggs
eggs
call 2 to eggs
eggs


In [97]:
class DescState:  
    def __init__(self, value):  
        self.value = value  
    def __get__(self, instance, owner):  
        return self.value * 10  # 返回描述符状态
    def __set__(self, instance, value):  
        self.value = value  

# Client Class  
class CalcAttr:  
    X = 2  # 共享X
    Y = 3  
    def __init__(self):  
        self._Z = None
    Z = DescState(10)

obj = CalcAttr()  
print(obj.X, obj.Y, obj.Z) # 100 3 5
obj.X = 5  
CalcAttr.Y = 6  
obj.Z = 7  
print(obj.X, obj.Y, obj.Z) # 50 6 7
obj2 = CalcAttr()  
print(obj2.X, obj2.Y, obj2.Z) # 50 6 7


2 3 100
5 6 70
2 6 70


简化版本

In [168]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
        
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print(f'call {self.calls} to {self.func.__name__}')
        print(f'run {self.func.__name__}({args}, {kwargs})')
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner):
        print('get')
        print(f'{self.__class__.__name__}, {instance.__class__.__name__}, {owner.__class__.__name__}')
        def wrapper(*args, **kwargs):
            print(f'wrapper return {self}({instance}, {args}, {kwargs})')
            return self(instance, *args, **kwargs) # 此时返回后 call调用的*args中包含了装饰的实例
        print('return')
        return wrapper
    

In [169]:
class C:
    @tracer
    def method1(self, x, y):
        print(x + y)

    @tracer
    def method2(self, x, y):
        print(x * y)

In [170]:
x = C()
x.method1(1, 2)
x.method2(10, 25)
x.method2(10, 2)
y = C()
y.method1(1, 5)
y.method2(10, 2)

get
tracer, C, type
return
wrapper return <__main__.tracer object at 0x111d508d0>(<__main__.C object at 0x111d38190>, (1, 2), {})
call 1 to method1
run method1((<__main__.C object at 0x111d38190>, 1, 2), {})
3
get
tracer, C, type
return
wrapper return <__main__.tracer object at 0x111d50990>(<__main__.C object at 0x111d38190>, (10, 25), {})
call 1 to method2
run method2((<__main__.C object at 0x111d38190>, 10, 25), {})
250
get
tracer, C, type
return
wrapper return <__main__.tracer object at 0x111d50990>(<__main__.C object at 0x111d38190>, (10, 2), {})
call 2 to method2
run method2((<__main__.C object at 0x111d38190>, 10, 2), {})
20
get
tracer, C, type
return
wrapper return <__main__.tracer object at 0x111d508d0>(<__main__.C object at 0x111d3a810>, (1, 5), {})
call 2 to method1
run method1((<__main__.C object at 0x111d3a810>, 1, 5), {})
6
get
tracer, C, type
return
wrapper return <__main__.tracer object at 0x111d50990>(<__main__.C object at 0x111d3a810>, (10, 2), {})
call 3 to method2
ru

In [159]:
@tracer  
def spam(a, b, c):  
    print(a + b + c)  
    

@tracer
def eggs():
    print('eggs')

spam(1, 2, 3)
spam('a', 'b', 'c')
eggs()
eggs()

call 1 to spam
run spam((1, 2, 3), {})
6
call 2 to spam
run spam(('a', 'b', 'c'), {})
abc
call 1 to eggs
run eggs((), {})
eggs
call 2 to eggs
run eggs((), {})
eggs


In [172]:
def tracer2(func):
    """可用于普通函数和类一级方法"""
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print(f'call {calls} to {func.__name__}')
        return func(*args, **kwargs)
    return wrapper


class C:
    @tracer2
    def method1(self, x, y):
        print(x + y)

    @tracer2
    def method2(self, x, y):
        print(x * y)


x = C()
x.method1(1, 2)
x.method2(10, 25)
x.method2(10, 2)
y = C()
y.method1(1, 5)
y.method2(10, 2)

call 1 to method1
3
call 1 to method2
250
call 2 to method2
20
call 2 to method1
6
call 3 to method2
20
